# Make interactive country map for geographical variations of PFDs

Coroners areas are not the same as geographical areas. Might be difficult

#Approximating 42 areas from police force area

https://geoportal.statistics.gov.uk/datasets/ons::police-force-areas-december-2016-full-extent-boundaries-in-england-and-wales-1/about

Data from json file of police boundaries. 43 regions

NHS data took absolutely ages and wont finish downloading

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
!pip install geopandas
import geopandas as gpd

In [ ]:
boundary = gpd.read_file('Police_Boundaries_England_and_Wales.geojson')

In [ ]:
display(boundary.head())
display(boundary.shape)

Check what the columns are

Rename columns

In [ ]:
boundary.rename(columns = {'PFA16NM':'Area'}, inplace = True)

Find the number of unique columns in the regions column of england and wales and display them

Stack two dataframe together

In [ ]:
area_frequency = pd.read_csv('geographical_variation_2022-06-22.csv')
display(area_frequency.head())

Next step is to match "coroner area" with england-wales police areas "Area", which is the hard part

Loop through coroner area, find the number of rows in police area that contains 

South and West Wales is problematic, because the boundaries are completely difference

Need to merge police areas and coroner areas together

In [ ]:
boundary["dissolve"] = np.nan

In [ ]:
dissolve_areas = ['Dyfed-Powys', 'South Wales']
for i in boundary.index: 
  if boundary.loc[i, 'Area'] not in dissolve_areas: 
    boundary.loc[i, 'dissolve'] = i
  else: 
    boundary.loc[i, 'dissolve']=90

In [ ]:
boundary_1 = boundary.dissolve(by = 'dissolve')

In [ ]:
display(boundary_1[boundary_1.index==90])

In [ ]:
print(boundary_1.shape)

In [ ]:
boundary_1.loc[90, 'Area'] = 'South West Wales'

Success! I've managed to merge the police area into one

Next, try to merge the 4 coroner areas into 1

In [ ]:
area_frequency_1 = area_frequency

In [ ]:
area_frequency_1.loc[85, 'coroner area']= 'South West Wales' 
south = area_frequency_1.loc[area_frequency_1['coroner area']=='South Wales Central', 'Number of reports'].values[0]
cere = area_frequency_1.loc[area_frequency_1['coroner area']=='Ceredigion', 'Number of reports'].values[0]
carpem = area_frequency_1.loc[area_frequency_1['coroner area']=='Carmarthenshire and Pembrokeshire', 'Number of reports'].values[0]
swan = area_frequency_1.loc[area_frequency_1['coroner area']=='Swansea and Neath Port Talbot', 'Number of reports'].values[0]

area_frequency_1.loc[85, 'Number of reports'] = south + cere + carpem + swan

In [ ]:
area_frequency_1 = area_frequency_1[area_frequency_1['coroner area']!='South Wales Central']
area_frequency_1 = area_frequency_1[area_frequency_1['coroner area']!='Ceredigion']
area_frequency_1 = area_frequency_1[area_frequency_1['coroner area']!='Carmarthenshire and Pembrokeshire']
area_frequency_1 = area_frequency_1[area_frequency_1['coroner area']!='Swansea and Neath Port Talbot']

In [ ]:
print(area_frequency_1.shape)

Merge Metropolitan police and "City of London"

Merge in the boundary

In [ ]:
boundary_1 = boundary_1.reset_index()

In [ ]:
display(boundary_1.head())

In [ ]:
boundary_1.loc[boundary_1['Area']=='Metropolitan Police', 'dissolve']=100
boundary_1.loc[boundary_1['Area']=='City of London', 'dissolve']=100
boundary_1.set_index('dissolve', inplace = True)


In [ ]:
boundary_1 = boundary_1.dissolve(by = 'dissolve')

In [ ]:
display(boundary_1)

In [ ]:
boundary_1.loc[100, 'Area'] = 'London'

In [ ]:
print(boundary_1.shape)

Now try to merge coroner areas - everything containing London will go under Metropolitan police. Forget City of London

In [ ]:
print(area_frequency_1.shape)

In [ ]:
area_frequency_1.loc[86, 'coroner area'] = 'London'
area_frequency_1.loc[86, 'Number of reports'] = area_frequency_1.loc[area_frequency_1['coroner area'].str.contains(' London'), 'Number of reports'].sum()

In [ ]:
print(area_frequency_1.shape)

In [ ]:
display(area_frequency_1.loc[85:86, :])

In [ ]:
#df_1 = area_frequency['coroner area'].str.contains('London')
df_1 = area_frequency_1.loc[area_frequency_1['coroner area'].str.contains(' London'), :]
display(df_1)
print(df_1.shape)

In [ ]:
index_to_remove = df_1.index
print(index_to_remove)

In [ ]:
for index in index_to_remove: 
  area_frequency_1.drop(index, inplace = True)

In [ ]:
print(area_frequency_1.shape)

Sucess

Now try to match and police area and coroner area

In [ ]:
area_frequency_1["police_area"] = np.nan

Do a simple filter first: if the two names are identical, match them

In [ ]:
import re

In [ ]:
for i in area_frequency_1.index: 
  for j in boundary_1.index: 
    coroner_area=re.sub("\(.*?\)","",area_frequency_1.loc[i, 'coroner area'])
    if coroner_area == boundary_1.loc[j, 'Area']: 
      # If the two areas are spelt identically, I can just match them
      area_frequency_1.loc[i, 'police_area']=area_frequency_1.loc[i, 'coroner area']
    elif coroner_area in boundary_1.loc[j, 'Area']: 
      # if the coroner area name is contained in the police area name, I can also match them (hopefully)
      # it turns out this elif statement is useless
      area_frequency_1.loc[i, 'police_area']=boundary_1.loc[j, 'Area']
    elif boundary_1.loc[j, 'Area'] in coroner_area: 
      # if the coroner area name contains police area. Print it out and I will check
      # I have checked that the coroner areas are actually all conatined in the polie area, so I will also maatch them
      area_frequency_1.loc[i, 'police_area'] = boundary_1.loc[j, 'Area']

In [ ]:
print(area_frequency_1['police_area'].isnull().sum())

In [ ]:
df = area_frequency_1[area_frequency_1['police_area'].notna()]
print(df.shape)

I've now sorted 42 out of 82. 33/75 remaining

This is dumb but I'm just going to match manually from this point onwards. There are 40 remaining. Some of them are not obviously a match at all and I don't see how I can do it with code

In [ ]:
# first item on the sublist is coroner area, second item is police area
# This is got to be manually done. There are 40 of them
replacements = [['Wiltshire and Swindon', 'Wiltshire'], 
               ['Cornwall and Isles of Scilly','Devon and Cornwall' ], 
               ['Exeter and Greater Devon', 'Devon and Cornwall'], 
               ['Plymouth, Torbay and South Devon', 'Devon and Cornwall'], 
               ['Avon', 'Avon and Somerset'], 
               ['Somerset', 'Avon and Somerset'], 
               ['Brighton and Hove', 'Sussex'],
               ['Manchester South', 'Greater Manchester'],
               ['Manchester North', 'Greater Manchester'],
               ['Manchester West', 'Greater Manchester'],
               ['Manchester City', 'Greater Manchester'],
               ['Birmingham and Solihull', 'West Midlands'],
               ['Black Country', 'West Midlands'],
               ['Milton Keynes', 'Thames Valley'],
               ['Liverpool and the Wirral', 'Merseyside'],
               ['Worcestershire', 'West Mercia'],
               ['Sunderland', 'Northumbria'],
               ['Blackpool and the Fylde', 'Lancashire'],
               ['Berkshire', 'Thames Valley'],
               ['Oxfordshire', 'Thames Valley'],
               ['Gwent', 'Gwent'],
               ['Shropshire, Telford and Wrekin', 'West Mercia'],
               ['Coventry', 'West Midlands'],
               ['Isle of Wight', 'Hampshire'],
               ['Buckinghamshire', 'Thames Valley'],
               ['Newcastle upon Tyne', 'Northumbria'],
               ['Teesside', 'Cleveland'],
                ['Hartlepool', 'Cleveland'], 
               ['North Northumberland', 'Northumbria'],
               ['South Northumberland', 'Northumbria'],
               ['North West Wales', 'North Wales'],
               ['East Riding and Hull', 'Humberside'],
               ['Gateshead and South Tyneside', 'Northumbria'],
               ['North Tyneside', 'Northumbria'],
               ['York City', 'North Yorkshire'],
               ['Sefton, Knowsley and St Helens', 'Merseyside'], 
                ['Herefordshire', 'West Mercia']
               ]
for replacement in replacements: 
  area_frequency_1.loc[area_frequency_1['coroner area']==replacement[0], 'police_area'] =replacement[1]


In [ ]:
display(area_frequency_1[area_frequency_1['police_area'].isna()])

In [ ]:
display(area_frequency_1.head())

In [ ]:
display(area_frequency_1[area_frequency_1['police_area']=='London'])

# Find out how many police areas take 1, 2, 3..4 coroner areas. 

In [ ]:
freq = area_frequency_1.police_area.value_counts().reset_index()

In [ ]:
display(freq[freq['index'].str.contains('London')])

In [ ]:
freq.rename(columns = {'index': 'police area', 'police_area': 'number of coronial area mergers'}, inplace = True)
freq.loc[freq['police area']=='London', 'police area'] = 'London'
freq.loc[freq['police area']=='London', 'number of coronial area mergers'] = 8
freq.loc[freq['police area']=='South West Wales', 'number of coronial area mergers'] = 4

In [ ]:
freq = freq.sort_values(by = 'number of coronial area mergers', ascending = False)

In [ ]:
display(freq)

In [ ]:
freq.to_csv('number_of_coronial_merger.csv')

#Now I can finally merge the boundary_1 and area_frequency_1 together

Merge the two dataframe with the merge (how = 'left')

In [ ]:
display(boundary_1.keys())

In [ ]:
boundary_1.rename(columns = {'Area': 'police_area'}, inplace = True)

In [ ]:
area_boundary = area_frequency_1.merge( boundary_1, on = 'police_area', how = 'left')

In [ ]:
display(area_boundary[area_boundary.isnull().any(axis=1)])

No NAN values and missing areas, good

In [ ]:
area_boundary.drop(columns = ['coroner area', 'OBJECTID', 'Unnamed: 0'], inplace = True)

Aggregate rows with the same Police area together, keeping one copy of the longitude and latitude, summing the number of reports

In [ ]:
area_boundary_1 = area_boundary[['Number of reports', 'police_area']]
area_boundary_2 = area_boundary[['police_area', 'geometry', 'PFA16CD', 'BNG_E', 'BNG_N', 'LONG', "LAT", 'Shape__Area', "Shape__Length"]]

In [ ]:
df_3 = area_boundary_1.groupby(by = 'police_area').sum()
df_3.reset_index(inplace = True)
display(df_3.shape)

In [ ]:
df_4 = area_boundary_2.drop_duplicates()
display(df_4.shape)

In [ ]:
area_boundary = df_3.merge(df_4, on = 'police_area', how = 'left')

In [ ]:
display(area_boundary.shape)
display(area_boundary[area_boundary.isnull().any(axis=1)])

Success in merging and calculating the total number of reports of each police area

#Now actually plot the map

In [ ]:
import pandas as pd
from geopandas import GeoDataFrame

In [ ]:
area_boundary_geo = GeoDataFrame(area_boundary, crs="EPSG:4326", geometry='geometry')

In [ ]:
fig, ax = plt.subplots(1, figsize=(18, 18))
plt.xticks(ticks =[], labels = None)
plt.yticks(ticks =[], labels = None)
map = area_boundary_geo.plot(column = 'Number of reports', figsize = (18, 18), ax = ax, legend = True, cmap = 'Oranges')
for x, y, area in zip(area_boundary_geo['LONG'], area_boundary_geo['LAT'], area_boundary_geo['police_area']): 
  map.annotate(area, xy = (x, y))

In [ ]:
fig.savefig('geographical_variations_orange.png')